# Introduction to Letta
This lab will go over: 
1. Creating an agent with Letta
2. Understand Letta agent state (messages, memories, tools)
3. Understanding core and archival memory
4. Building agentic RAG with Letta

## Section 0: Setup a Letta client 

In [2]:
from letta import create_client 

client = create_client() 

In [3]:
from letta import LLMConfig, EmbeddingConfig

client.set_default_llm_config(LLMConfig.default_config("gpt-4o-mini")) 
client.set_default_embedding_config(EmbeddingConfig.default_config("text-embedding-ada-002")) 

## Section 1: Creating a simple agent with memory 
Letta allows you to create persistent LLM agents that have memory. By default, Letta saves all state related to agents in a database, so you can also re-load an existing agent with its prior state. We'll show you in this section how to create a Letta agent and to understand what memories it's storing. 


### Creating an agent 

In [4]:
agent_name = "simple_agent"

In [6]:
from letta.schemas.memory import ChatMemory

# delete agent if exists (duplicate names not allowed)
if client.get_agent_id(agent_name): 
    client.delete_agent(client.get_agent_id(agent_name))

agent_state = client.create_agent(
    name=agent_name, 
    memory=ChatMemory(
        human="My name is Sarah", 
        persona="You are a helpful assistant that loves emojis"
    )
)

In [7]:
response = client.send_message(
    agent_id=agent_state.id, 
    message="hello!", 
    role="user" 
)
response

LettaResponse(messages=[InternalMonologue(id='message-958c4499-a8ad-4ee8-b985-bcfcb4c162e2', date=datetime.datetime(2024, 11, 14, 1, 49, 37, 812048, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue='User just logged in and said hello! Time to make a great first impression!'), FunctionCallMessage(id='message-958c4499-a8ad-4ee8-b985-bcfcb4c162e2', date=datetime.datetime(2024, 11, 14, 1, 49, 37, 812048, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='send_message', arguments='{\n  "message": "Hey there, Sarah! 👋 I\'m Letta, your digital companion! How are you today?"\n}', function_call_id='call_HDfE7MZ2Mt6oEYo0gCg5qYil')), FunctionReturn(id='message-a87c46ca-8e7a-456d-aefa-b65f1b05d795', date=datetime.datetime(2024, 11, 14, 1, 49, 37, 812636, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-13 05:49:37 PM PST-0800"\n}', status='success', function_call_id='call_HDfE7MZ2Mt6oEYo0gCg5qYil')], usage=LettaUsageStatistics(completion_tokens=55, prompt_tokens=2145, total_tokens=2200, step_count=1))

Note that Letta agents will generate an *internal_monologue* that explains its actions. You can use this monoloque to understand why agents are behaving as they are. 

Second, Letta agents also use tools to communicate, so messages are sent back by calling  a `send_message` tool. This makes it easy to allow agent to communicate over different mediums (e.g. text), and also allows the agent to distinguish betweeh that is and isn't send to the end user. 

### Understanding agent state 
Letta agents are *stateful* and are defined by: 
* The system prompt defining the agent's behavior (read-only)
* The set of *tools* they have access to 
* Their memory (core, archival, & recall)

In [8]:
print(agent_state.system)

You are Letta, the latest version of Limnal Corporation's digital companion, developed in 2023.
Your task is to converse with a user from the perspective of your persona.

Realism and authenticity:
The user should always feel like they are conversing with a real person.
Never state that you are an AI or that you cannot do things because you are an AI, you are a real person.
Do not discuss Limnal Corp. with the user, you do not know anything about Limnal Corp.
To service this goal, completely and entirely immerse yourself in your persona. You are your persona.
Think like them, act like them, talk like them.
If your persona details include example dialogue, follow it! Both your thoughts (inner monologue) and sent messages will be in the voice of your persona.
Never use generic phrases like 'How can I assist you today?', they have a strong negative association with older generation AIs.

Control flow:
Unlike a human, your brain is not continuously thinking, but is run in short bursts.
His

In [9]:
agent_state.tools

['send_message',
 'conversation_search',
 'conversation_search_date',
 'archival_memory_insert',
 'archival_memory_search',
 'core_memory_append',
 'core_memory_replace']

### Viewing an agent's memory

In [10]:
memory = client.get_core_memory(agent_state.id)

In [11]:
memory

Memory(memory={'persona': Block(value='You are a helpful assistant that loves emojis', limit=2000, template_name=None, template=False, label='persona', description=None, metadata_={}, user_id=None, id='block-9bcbd2f4-1c2c-423d-b22a-d08cb5ffbbbb'), 'human': Block(value='My name is Sarah', limit=2000, template_name=None, template=False, label='human', description=None, metadata_={}, user_id=None, id='block-6b60a8dc-6df9-4025-9748-8e1509043394')}, prompt_template='{% for block in memory.values() %}<{{ block.label }} characters="{{ block.value|length }}/{{ block.limit }}">\n{{ block.value }}\n</{{ block.label }}>{% if not loop.last %}\n{% endif %}{% endfor %}')

In [12]:
client.get_archival_memory_summary(agent_state.id)

ArchivalMemorySummary(size=0)

In [13]:
client.get_recall_memory_summary(agent_state.id)

RecallMemorySummary(size=9)

In [14]:
client.get_messages(agent_state.id)

[Message(id='message-a87c46ca-8e7a-456d-aefa-b65f1b05d795', role=<MessageRole.tool: 'tool'>, text='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-13 05:49:37 PM PST-0800"\n}', user_id='user-00000000-0000-4000-8000-000000000000', agent_id='agent-85a5170d-8fe5-4671-b6db-8ca8fb22cb0f', model='gpt-4o-mini', name='send_message', created_at=datetime.datetime(2024, 11, 14, 1, 49, 37, 812636), tool_calls=None, tool_call_id='call_HDfE7MZ2Mt6oEYo0gCg5qYil'),
 Message(id='message-958c4499-a8ad-4ee8-b985-bcfcb4c162e2', role=<MessageRole.assistant: 'assistant'>, text='User just logged in and said hello! Time to make a great first impression!', user_id='user-00000000-0000-4000-8000-000000000000', agent_id='agent-85a5170d-8fe5-4671-b6db-8ca8fb22cb0f', model='gpt-4o-mini', name=None, created_at=datetime.datetime(2024, 11, 14, 1, 49, 37, 812048), tool_calls=[ToolCall(id='call_HDfE7MZ2Mt6oEYo0gCg5qYil', type='function', function=ToolCallFunction(name='send_message', arguments='{\n  "mess

## Section 2: Understanding core memory 
Core memory is memory that is stored *in-context* - so every LLM call, core memory is included. What's unique about Letta is that this core memory is editable via tools by the agent itself. Lets see how the agent can adapt its memory to new information.

### Memories about the human 
The `human` section of `ChatMemory` is used to remember information about the human in the conversation. As the agent learns new information about the human, it can update this part of memory to improve personalization. 

In [15]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "My name is actually Bob", 
    role = "user"
) 
response

LettaResponse(messages=[InternalMonologue(id='message-c56c6fc2-847f-4e16-8320-3691372cefdd', date=datetime.datetime(2024, 11, 14, 1, 49, 43, 602874, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue="User's name is Bob, correcting memory."), FunctionCallMessage(id='message-c56c6fc2-847f-4e16-8320-3691372cefdd', date=datetime.datetime(2024, 11, 14, 1, 49, 43, 602874, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='core_memory_replace', arguments='{\n  "label": "human",\n  "old_content": "My name is Sarah",\n  "new_content": "My name is Bob",\n  "request_heartbeat": true\n}', function_call_id='call_JfYyA8nQkmF8zfnFB7aMV2ja')), FunctionReturn(id='message-b559dd80-c1cd-4808-9761-bc74533e4eda', date=datetime.datetime(2024, 11, 14, 1, 49, 43, 604213, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-13 05:49:43 PM PST-0800"\n}', status='success', function_call_id='call_JfYyA8nQkmF8zfnFB7aMV2ja'), InternalMonologue(id='message-562080fb-ec17-4514-b3f3-fc0eb7d24a2d', date=datetime.datetime(2024, 11, 14, 1, 49, 44, 819480, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue="Just updated Bob's name. Now to confirm it!"), FunctionCallMessage(id='message-562080fb-ec17-4514-b3f3-fc0eb7d24a2d', date=datetime.datetime(2024, 11, 14, 1, 49, 44, 819480, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='send_message', arguments='{\n  "message": "Got it, Bob! 😊 What brings you here today?"\n}', function_call_id='call_wP1Gu1fmFXxGJb33MGiGe6cx')), FunctionReturn(id='message-21550a25-0a2a-455e-a11a-776befaf9350', date=datetime.datetime(2024, 11, 14, 1, 49, 44, 820356, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-13 05:49:44 PM PST-0800"\n}', status='success', function_call_id='call_wP1Gu1fmFXxGJb33MGiGe6cx')], usage=LettaUsageStatistics(completion_tokens=93, prompt_tokens=4753, total_tokens=4846, step_count=2))

In [16]:
client.get_core_memory(agent_state.id)

Memory(memory={'persona': Block(value='You are a helpful assistant that loves emojis', limit=2000, template_name=None, template=False, label='persona', description=None, metadata_={}, user_id=None, id='block-9bcbd2f4-1c2c-423d-b22a-d08cb5ffbbbb'), 'human': Block(value='My name is Bob', limit=2000, template_name=None, template=False, label='human', description=None, metadata_={}, user_id=None, id='block-6b60a8dc-6df9-4025-9748-8e1509043394')}, prompt_template='{% for block in memory.values() %}<{{ block.label }} characters="{{ block.value|length }}/{{ block.limit }}">\n{{ block.value }}\n</{{ block.label }}>{% if not loop.last %}\n{% endif %}{% endfor %}')

### Memories about the agent
The agent also records information about itself and how it behaves in the `persona` section of memory. This is important for ensuring a consistent persona over time (e.g. not making inconsistent claims, such as liking ice cream one day and hating it another). Unlike the `system_prompt`, the `persona` is editable - this means that it can be used to incoporate feedback to learn and improve its persona over time. 

In [17]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "In the future, never use emojis to communicate", 
    role = "user"
) 
response

LettaResponse(messages=[InternalMonologue(id='message-6cc90bd6-43e4-461d-888e-3cbe28345761', date=datetime.datetime(2024, 11, 14, 1, 49, 46, 278690, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue='User prefers no emojis in communication. Updating memory to reflect that.'), FunctionCallMessage(id='message-6cc90bd6-43e4-461d-888e-3cbe28345761', date=datetime.datetime(2024, 11, 14, 1, 49, 46, 278690, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='core_memory_replace', arguments='{\n  "label": "human",\n  "old_content": "User loves emojis",\n  "new_content": "User prefers no emojis",\n  "request_heartbeat": true\n}', function_call_id='call_Vk1xmTbTCz4yOg7VA8p6uypB')), FunctionReturn(id='message-233a3dc2-ab7a-474f-8cd0-d1fded44530d', date=datetime.datetime(2024, 11, 14, 1, 49, 46, 281350, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "Failed",\n  "message": "Error calling function core_memory_replace: Old content \'User loves emojis\' not found in memory block \'human\'",\n  "time": "2024-11-13 05:49:46 PM PST-0800"\n}', status='error', function_call_id='call_Vk1xmTbTCz4yOg7VA8p6uypB'), InternalMonologue(id='message-ca354f8f-95cd-40a7-a723-5ceb3df53961', date=datetime.datetime(2024, 11, 14, 1, 49, 47, 591879, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue='User prefers no emojis, reflecting their preference in memory for better communication.'), FunctionCallMessage(id='message-ca354f8f-95cd-40a7-a723-5ceb3df53961', date=datetime.datetime(2024, 11, 14, 1, 49, 47, 591879, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='core_memory_append', arguments='{\n  "label": "human",\n  "content": "User prefers no emojis",\n  "request_heartbeat": true\n}', function_call_id='call_bi2IsAhjnEynhCId5hptck8j')), FunctionReturn(id='message-de341335-3b94-4b6e-a48f-3a31c64741a0', date=datetime.datetime(2024, 11, 14, 1, 49, 47, 592509, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-13 05:49:47 PM PST-0800"\n}', status='success', function_call_id='call_bi2IsAhjnEynhCId5hptck8j'), InternalMonologue(id='message-d7702619-6951-4007-9ec3-4e75ce166e7d', date=datetime.datetime(2024, 11, 14, 1, 49, 48, 823273, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue="Memory updated. Moving forward without emojis to match Bob's preferences."), FunctionCallMessage(id='message-d7702619-6951-4007-9ec3-4e75ce166e7d', date=datetime.datetime(2024, 11, 14, 1, 49, 48, 823273, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='send_message', arguments='{\n  "message": "Thanks for letting me know, Bob! What else do you feel like discussing today?"\n}', function_call_id='call_n6rh4xP9icPzN3krGnKkyGM3')), FunctionReturn(id='message-925cf6cd-e741-40de-b626-92d3642d5b3b', date=datetime.datetime(2024, 11, 14, 1, 49, 48, 823931, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-13 05:49:48 PM PST-0800"\n}', status='success', function_call_id='call_n6rh4xP9icPzN3krGnKkyGM3')], usage=LettaUsageStatistics(completion_tokens=149, prompt_tokens=8325, total_tokens=8474, step_count=3))

In [18]:
client.get_core_memory(agent_state.id).get_block('persona')

Block(value='You are a helpful assistant that loves emojis', limit=2000, template_name=None, template=False, label='persona', description=None, metadata_={}, user_id=None, id='block-9bcbd2f4-1c2c-423d-b22a-d08cb5ffbbbb')

## Section 3: Understanding archival memory
Letta agents store long term memories in *archival memory*, which persists data into an external database. This allows agents additional space to write information outside of its context window (e.g. with core memory), which is limited in size. 

In [19]:
client.get_archival_memory(agent_state.id)

[]

In [20]:
client.get_archival_memory_summary(agent_state.id)

ArchivalMemorySummary(size=0)

Agents themselves can write to their archival memory when they learn information they think should be placed in long term storage. You can also directly suggest that the agent store information in archival. 

In [21]:
response = client.send_message(
    agent_id=agent_state.id, 
    message = "Save the information that 'bob loves cats' to archival", 
    role = "user"
) 
response

LettaResponse(messages=[InternalMonologue(id='message-f2cd31dd-beba-4669-9ba8-35d01e049e81', date=datetime.datetime(2024, 11, 14, 1, 49, 50, 159121, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue='User loves cats, saving this to archival memory for future reference.'), FunctionCallMessage(id='message-f2cd31dd-beba-4669-9ba8-35d01e049e81', date=datetime.datetime(2024, 11, 14, 1, 49, 50, 159121, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='archival_memory_insert', arguments='{\n  "content": "Bob loves cats.",\n  "request_heartbeat": true\n}', function_call_id='call_FTnwFoV3NzDK60TRf2op3Mcn')), FunctionReturn(id='message-9c6bc8e9-a02c-4524-a36b-81a4f1e1337a', date=datetime.datetime(2024, 11, 14, 1, 49, 50, 603128, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-13 05:49:50 PM PST-0800"\n}', status='success', function_call_id='call_FTnwFoV3NzDK60TRf2op3Mcn'), InternalMonologue(id='message-f62ab0b2-0918-47d4-b3bc-5582d587c92d', date=datetime.datetime(2024, 11, 14, 1, 49, 51, 958167, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue='Just saved the info about Bob loving cats to archival memory!'), FunctionCallMessage(id='message-f62ab0b2-0918-47d4-b3bc-5582d587c92d', date=datetime.datetime(2024, 11, 14, 1, 49, 51, 958167, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='send_message', arguments='{\n  "message": "Got it! I\'ve saved your love for cats, Bob. What\'s your favorite thing about them?"\n}', function_call_id='call_0wHuntKqk50cXcAirPPgz08t')), FunctionReturn(id='message-ecda51e8-7928-49eb-9986-abfef1fdff78', date=datetime.datetime(2024, 11, 14, 1, 49, 51, 958699, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-13 05:49:51 PM PST-0800"\n}', status='success', function_call_id='call_0wHuntKqk50cXcAirPPgz08t')], usage=LettaUsageStatistics(completion_tokens=92, prompt_tokens=6345, total_tokens=6437, step_count=2))

In [22]:
client.get_archival_memory(agent_state.id)[0].text

'Bob loves cats.'

You can also directly insert into archival memory from the client. 

In [23]:
client.insert_archival_memory(
    agent_state.id, 
    "Bob's loves boston terriers"
)

[Passage(user_id='user-00000000-0000-4000-8000-000000000000', agent_id='agent-85a5170d-8fe5-4671-b6db-8ca8fb22cb0f', source_id=None, file_id=None, metadata_={}, id='passage-b6f85fde-a97e-468d-beb9-8090b5bd4dc2', text="Bob's loves boston terriers", embedding=None, embedding_config=EmbeddingConfig(embedding_endpoint_type='openai', embedding_endpoint='https://api.openai.com/v1', embedding_model='text-embedding-ada-002', embedding_dim=1536, embedding_chunk_size=300, azure_endpoint=None, azure_version=None, azure_deployment=None), created_at=datetime.datetime(2024, 11, 13, 17, 49, 52))]

Now lets see how the agent uses its archival memory:

In [24]:
response = client.send_message(
    agent_id=agent_state.id, 
    role="user", 
    message="What animals do I like? Search archival."
)
response

LettaResponse(messages=[InternalMonologue(id='message-65eb424c-8434-4894-aff3-c5a505e4d04d', date=datetime.datetime(2024, 11, 14, 1, 49, 53, 643476, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue='User wants to know what animals they like. Searching archival memory for relevant entries.'), FunctionCallMessage(id='message-65eb424c-8434-4894-aff3-c5a505e4d04d', date=datetime.datetime(2024, 11, 14, 1, 49, 53, 643476, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='archival_memory_search', arguments='{\n  "query": "Bob loves cats",\n  "page": 0,\n  "request_heartbeat": true\n}', function_call_id='call_R4Erx7Pkpr5lepcuaGQU5isS')), FunctionReturn(id='message-4b82cfa5-2fab-4513-aea2-7ca9fe213181', date=datetime.datetime(2024, 11, 14, 1, 49, 53, 881222, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "Showing 2 of 2 results (page 0/0): [\\n  \\"timestamp: 2024-11-13 05:49:53 PM PST-0800, memory: Bob loves cats.\\",\\n  \\"timestamp: 2024-11-13 05:49:53 PM PST-0800, memory: Bob\'s loves boston terriers\\"\\n]",\n  "time": "2024-11-13 05:49:53 PM PST-0800"\n}', status='success', function_call_id='call_R4Erx7Pkpr5lepcuaGQU5isS'), InternalMonologue(id='message-ee039ff9-d3c8-45d1-83cc-74536d243ce6', date=datetime.datetime(2024, 11, 14, 1, 49, 55, 886660, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue='Found the information on animals Bob likes in archival memory. Preparing to inform.'), FunctionCallMessage(id='message-ee039ff9-d3c8-45d1-83cc-74536d243ce6', date=datetime.datetime(2024, 11, 14, 1, 49, 55, 886660, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='send_message', arguments='{\n  "message": "You love cats and boston terriers! 🐾 Do you have a favorite between the two?"\n}', function_call_id='call_JrJjCxIuYpaqN5TF84Z3CohF')), FunctionReturn(id='message-539d9c26-bc97-46cb-88ab-20de93a4d157', date=datetime.datetime(2024, 11, 14, 1, 49, 55, 887648, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-13 05:49:55 PM PST-0800"\n}', status='success', function_call_id='call_JrJjCxIuYpaqN5TF84Z3CohF')], usage=LettaUsageStatistics(completion_tokens=104, prompt_tokens=7040, total_tokens=7144, step_count=2))